## Запись данных наблюдений

In [6]:
import csv
from datetime import datetime
import psycopg2

In [19]:
#!!!
def get_datasets():
    datasets = {
        # 'Неман-Белица': 'Неман',
        # 'Неман-Гродно': 'Неман',
        # 'Неман-Мосты': 'Неман',
        # 'Неман-Столбцы': 'Неман',

        # 'Вилия-Стешицы': 'Вилия',
        # 'Вилия-Михалишки': 'Вилия',

        # 'ЗападнаяДвина-Сураж': 'ЗападнаяДвина-А',
        # 'ЗападнаяДвина-Верхнедвинск': 'ЗападнаяДвина-А',
        # 'ЗападнаяДвина-Витебск': 'ЗападнаяДвина-Б',
        # 'ЗападнаяДвина-Полоцк': 'ЗападнаяДвина-Б',

        # 'Сож-Кричев': 'Сож',
        'Сож-Славгород': 'Сож',
        'Сож-Гомель': 'Сож'
    }
    return datasets

In [20]:
get_datasets()

{'Сож-Славгород': 'Сож', 'Сож-Гомель': 'Сож'}

### Функция выборки гидропостов и их кодов из БД

In [21]:
def get_stations_db():
    db_params = {
        "host": "192.168.29.134",
        "database": "pkogo",
        "user": "pkogouser",
        "password": "pkogouser",
        "port": "5432"
    }
    try:
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        print(cursor)
        print("DB connection works")
    except (Exception, psycopg2.Error) as error:
        print(f"Error connecting to database: {error}")


    sql = "SELECT station, station_id FROM maxlevel.STATIONS"
    cursor.execute(sql)
    result = cursor.fetchall()
    # print(result)

    if conn:
        cursor.close()
        conn.close()

    return {station: id for station, id in result}

In [22]:
get_stations_db()

<cursor object at 0x000001C14414F680; closed: 0>
DB connection works


{'Полоцк': 73122,
 'Столбцы': 74014,
 'Витебск': 73115,
 'Мосты': 74024,
 'Гродно': 74032,
 'Стешицы': 74198,
 'Михалишки': 74205,
 'Верхнедвинск': 73131,
 'Кричев': 79247,
 'Славгород': 79250,
 'Гомель': 79256,
 'Сураж': 73111,
 'Белица': 74021}

#### Функция чтения набора данных

In [23]:
# !!!
def write_dataset_db(year, dataset, db_connection, db_cursor):
    # Путь к csv-файлу
    ds_dir = f'data/{year}/Train'    
    fname = f'{ds_dir}/{dataset}.csv'

    # Извлечение название гидропоста из названия датасета,
    # например датасет Вилия-Михалишки, тогда гидропост Михалишки
    station = dataset.split('-')[1]
    
    # Извлечение из БД названий станций и их id (словарь)
    stations = get_stations_db()
    station_id = stations[station]
    
    print(f'fname==\n{fname}')
    with open(fname, newline='') as f: 
        reader = csv.DictReader(f, delimiter=';')
        # Прочитать заголовки
        fieldnames_list = ['station_id'] + reader.fieldnames
        fieldnames = ', '.join(fieldnames_list).replace('year', 'obs_year')
        print(f'fieldnames==\n{fieldnames}')

               
        # Создать плейсхолдер для sql
        placeholders = ', '.join([f'%({c})s' for c in fieldnames_list])
        print(f'placeholders==\n{placeholders}')

        sql = f"""
        INSERT INTO maxlevel.observations ({fieldnames}) 
        VALUES ({placeholders})
        """
        print(f'sql==\n{sql}')

        ## Удаление из таблицы наблюдений по данному датасету
        db_cursor.execute(f"DELETE FROM maxlevel.observations WHERE station_id={station_id}")
        db_connection.commit()
        
        ## Запись в таблицу Observations
        for row in reader:
            # Преобразование строк-дат в datetime
            for k in row:
                if 'd_' in k:
                    row[k] = datetime.strptime(row[k], '%d.%m.%Y')
                else:
                    row[k] = float(row[k])
            print(f'row==\n{row}')
            db_row = {'station_id': station_id} | row
            print(f'db_row==\n{db_row}')
            db_cursor.execute(sql, db_row)

        db_connection.commit()


In [24]:
def write_observations_db(year):

    db_params = {
        "host": "192.168.29.134",
        "database": "pkogo",
        "user": "pkogouser",
        "password": "pkogouser",
        "port": "5432"
    }
    try:
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        print(cursor)
        print("DB connection works")
    except (Exception, psycopg2.Error) as error:
        print(f"Error connecting to database: {error}")
    
    # # Удаление ВСЕХ строк из таблицы
    # cursor.execute("DELETE FROM maxlevel.observations")
    # conn.commit()
    
    # Итерация по датасетам
    datasets = get_datasets()
    for ds in datasets:            
        write_dataset_db(year, ds, conn, cursor)

    if conn:
        cursor.close()
        conn.close()

In [25]:
_ = write_observations_db(2024) # Перезапись всех исходных данных (наблюдений) по всем гидропостам и рекам

<cursor object at 0x000001C14414F680; closed: 0>
DB connection works
<cursor object at 0x000001C14414F920; closed: 0>
DB connection works
fname==
data/2024/Train/Сож-Славгород.csv
fieldnames==
station_id, obs_year, d_0, S_2802, Smax, d_smax, H_2802, H_max, d_hmax, X, X1, X2, Xs, Q12, Q01, Q02, W12, W01, W02, Y12, Y01, Y02, Y_sum
placeholders==
%(station_id)s, %(year)s, %(d_0)s, %(S_2802)s, %(Smax)s, %(d_smax)s, %(H_2802)s, %(H_max)s, %(d_hmax)s, %(X)s, %(X1)s, %(X2)s, %(Xs)s, %(Q12)s, %(Q01)s, %(Q02)s, %(W12)s, %(W01)s, %(W02)s, %(Y12)s, %(Y01)s, %(Y02)s, %(Y_sum)s
sql==

        INSERT INTO maxlevel.observations (station_id, obs_year, d_0, S_2802, Smax, d_smax, H_2802, H_max, d_hmax, X, X1, X2, Xs, Q12, Q01, Q02, W12, W01, W02, Y12, Y01, Y02, Y_sum) 
        VALUES (%(station_id)s, %(year)s, %(d_0)s, %(S_2802)s, %(Smax)s, %(d_smax)s, %(H_2802)s, %(H_max)s, %(d_hmax)s, %(X)s, %(X1)s, %(X2)s, %(Xs)s, %(Q12)s, %(Q01)s, %(Q02)s, %(W12)s, %(W01)s, %(W02)s, %(Y12)s, %(Y01)s, %(Y02)s, %(Y_su